# Week13 Client Project: LSTM for Stock-like Time Series Prediction

**Full training (15 epochs)**

Train an LSTM on a synthetic stock-like series, save model, evaluate and produce simple report.

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import os

# Generate synthetic stock-like series (trend + seasonality + noise)
def generate_stock(n=2000, seed=1):
    np.random.seed(seed)
    t = np.arange(n)
    trend = 0.001 * t
    seasonal = 0.5*np.sin(0.02*t) + 0.2*np.sin(0.07*t)
    noise = 0.2*np.random.randn(n)
    price = 50 + trend + seasonal + noise
    return price

series = generate_stock(3000)
plt.figure(figsize=(10,3))
plt.plot(series[:500])
plt.title("Synthetic stock-like series (first 500 points)")
plt.show()

# Windowing
def make_windows(series, window_size=60, horizon=1):
    X, y = [], []
    for i in range(len(series)-window_size-horizon+1):
        X.append(series[i:i+window_size])
        y.append(series[i+window_size:i+window_size+horizon])
    return np.array(X), np.array(y).reshape(-1,)

window_size = 60
X, y = make_windows(series, window_size=window_size, horizon=1)

# Train/test split
split = int(len(X)*0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Scale (Standard)
scaler_X = StandardScaler()
nsamples, w = X_train.shape
X_train_s = scaler_X.fit_transform(X_train.reshape(-1,1)).reshape(nsamples,w)
X_test_s = scaler_X.transform(X_test.reshape(-1,1)).reshape(X_test.shape)

scaler_y = StandardScaler()
y_train_s = scaler_y.fit_transform(y_train.reshape(-1,1)).reshape(-1,)
y_test_s = scaler_y.transform(y_test.reshape(-1,1)).reshape(-1,)

# Build model
model = models.Sequential([
    layers.Input(shape=(window_size,1)),
    layers.LSTM(100, return_sequences=False),
    layers.Dense(50, activation='relu'),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

# reshape inputs
X_train_r = X_train_s.reshape((-1, window_size, 1))
X_test_r = X_test_s.reshape((-1, window_size, 1))

# Train
checkpoint = os.path.join("/mnt/data", "week13_client_best.h5")
callbacks = [tf.keras.callbacks.ModelCheckpoint(checkpoint, save_best_only=True, monitor='val_mae'),
             tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor='val_mae')]

history = model.fit(X_train_r, y_train_s, epochs=15, batch_size=64,
                    validation_data=(X_test_r, y_test_s), callbacks=callbacks)

# Evaluate
y_pred_s = model.predict(X_test_r).reshape(-1,)
y_pred = scaler_y.inverse_transform(y_pred_s.reshape(-1,1)).reshape(-1,)
y_true = y_test

mse = mean_squared_error(y_true, y_pred)
print("Test MSE:", mse)

plt.figure(figsize=(10,4))
plt.plot(y_true[:200], label='true')
plt.plot(y_pred[:200], label='pred')
plt.legend()
plt.title("Predicted vs True (first 200 test points)")
plt.show()

# Save model
out_model = os.path.join("/mnt/data", "week13_client_model.h5")
model.save(out_model)
print("Saved client model to", out_model)

# Simple evaluation report saved
report = f"Test MSE: {mse:.4f}\\nModel saved to {out_model}\\n"
with open(os.path.join("/mnt/data","week13_client_report.txt"), "w") as f:
    f.write(report)
print(report)
